In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [4]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [5]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [6]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [7]:
# Scale the dataset based on your needs (machine capacity) ideally ~500k rows
data_attack = data[data['Label'] == 1]
data_benign = data[data['Label'] == 0].sample(frac=0.02, random_state=13)
data = pd.concat([data_attack, data_benign], axis=0)
data = data.sample(frac=1, random_state=13).reset_index(drop=True)
data.Label.value_counts()

Label
1    95053
0    45904
Name: count, dtype: int64

In [8]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299,...,2299,2299,2299,2299,2299,2299,2299,2299,2299,2299
Backdoor,2169,2169,2169,2169,2169,2169,2169,2169,2169,2169,...,2169,2169,2169,2169,2169,2169,2169,2169,2169,2169
Benign,45904,45904,45904,45904,45904,45904,45904,45904,45904,45904,...,45904,45904,45904,45904,45904,45904,45904,45904,45904,45904
DoS,5794,5794,5794,5794,5794,5794,5794,5794,5794,5794,...,5794,5794,5794,5794,5794,5794,5794,5794,5794,5794
Exploits,31551,31551,31551,31551,31551,31551,31551,31551,31551,31551,...,31551,31551,31551,31551,31551,31551,31551,31551,31551,31551
Fuzzers,22310,22310,22310,22310,22310,22310,22310,22310,22310,22310,...,22310,22310,22310,22310,22310,22310,22310,22310,22310,22310
Generic,16560,16560,16560,16560,16560,16560,16560,16560,16560,16560,...,16560,16560,16560,16560,16560,16560,16560,16560,16560,16560
Reconnaissance,12779,12779,12779,12779,12779,12779,12779,12779,12779,12779,...,12779,12779,12779,12779,12779,12779,12779,12779,12779,12779
Shellcode,1427,1427,1427,1427,1427,1427,1427,1427,1427,1427,...,1427,1427,1427,1427,1427,1427,1427,1427,1427,1427


In [9]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [10]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [11]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [12]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [13]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
54790,59.166.0.7,149.171.126.7,3.814536e-08,4.108020e-08,0.000020,3.681363e-07,0.000123,4.908484e-07,2.369303e-08,5.817156e-08,...,0.000444,0.000355,4.588295e-08,4.588380e-08,4.191008e-08,4.191625e-08,0.0,4.300362e-08,"[3.814535708763979e-08, 4.108020165677007e-08,...",54790
62828,175.45.176.0,149.171.126.10,8.270785e-07,8.047329e-07,0.000125,2.403846e-06,0.000000,0.000000e+00,9.359471e-07,9.359471e-07,...,0.000000,0.000000,8.988155e-07,8.988322e-07,8.209898e-07,8.211105e-07,0.0,8.424114e-07,"[8.270784791926004e-07, 8.047329196686106e-07,...",62828
41042,59.166.0.5,149.171.126.2,2.390521e-09,2.574444e-09,0.000011,1.845651e-07,0.000124,1.922553e-07,1.484812e-09,1.547512e-09,...,0.000117,0.000056,1.120062e-10,1.183919e-10,2.626452e-09,2.626838e-09,0.0,2.694982e-09,"[2.3905211630149368e-09, 2.574444150982946e-09...",41042
80853,175.45.176.1,149.171.126.10,5.985176e-07,4.184611e-07,0.000125,1.250000e-06,0.000000,0.000000e+00,4.866925e-07,4.866925e-07,...,0.000000,0.000000,4.673841e-07,4.673928e-07,4.269147e-07,4.269775e-07,0.0,4.380540e-07,"[5.985176447375138e-07, 4.1846114668570904e-07...",80853
25170,175.45.176.2,149.171.126.10,4.521337e-09,3.598448e-09,0.000102,9.454226e-08,0.000002,3.636241e-08,7.242284e-09,7.237300e-09,...,0.000119,0.000119,6.992736e-09,7.003109e-09,4.967567e-09,4.968298e-09,0.0,3.810103e-09,"[4.521336912811051e-09, 3.5984476719847035e-09...",25170


In [14]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [15]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
# print(test_g)
test_g = test_g.to_directed()
# print(test_g)
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [17]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [18]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [19]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [20]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [21]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [22]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [23]:
# Convert to GPU
train_g = train_g.to('cuda')

In [24]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW_multiclass_v2.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [25]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_multiclass_v2.pkl'))

/tmp/ipykernel_124692/1257123129.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_UNSW_multiclass_v2.pkl'))


<All keys matched successfully>

In [26]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [27]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [28]:
# Convert to GPU
test_g = test_g.to('cuda')

In [29]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [30]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [31]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [32]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [33]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,0.027564,0.061195,0.106339,0.123702,0.088148,0.194936,0.051847,-0.021169,0.073368,0.141140,...,0.000000,0.0,0.002236,0.002236,1.167008e-08,1.192984e-08,9.322658e-08,9.324029e-08,0.0,9.565909e-08
1,0.027564,0.061195,0.106339,0.123702,0.088148,0.194936,0.051847,-0.021169,0.073368,0.141140,...,0.000000,0.0,0.001872,0.001872,9.275525e-08,9.296284e-08,7.805673e-08,7.806821e-08,0.0,8.009342e-08
2,0.027564,0.061195,0.106339,0.123702,0.088148,0.194936,0.051847,-0.021169,0.073368,0.141140,...,0.000000,0.0,0.000000,0.000000,5.564097e-07,5.564200e-07,5.082318e-07,5.083065e-07,0.0,5.214928e-07
3,0.027564,0.061195,0.106339,0.123702,0.088148,0.194936,0.051847,-0.021169,0.073368,0.141140,...,0.000002,0.0,0.002984,0.002984,1.430868e-07,1.456895e-07,1.243948e-07,1.244131e-07,0.0,1.276406e-07
4,0.027564,0.061195,0.106339,0.123702,0.088148,0.194936,0.051847,-0.021169,0.073368,0.141140,...,0.000000,0.0,0.003169,0.003169,1.915947e-07,1.915958e-07,1.321255e-07,1.321449e-07,0.0,1.355729e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84571,-0.022834,0.279163,-0.048518,0.017740,0.133633,0.538629,0.069336,-0.180243,-0.024673,0.098406,...,0.000000,0.0,0.000000,0.000000,1.741292e-07,1.741324e-07,1.590519e-07,1.590752e-07,0.0,1.632019e-07
84572,-0.012005,0.273036,-0.080949,0.021493,0.135640,0.547582,0.081690,-0.215371,-0.036512,0.086712,...,0.000000,0.0,0.000000,0.000000,1.589433e-07,1.589462e-07,1.451809e-07,1.452022e-07,0.0,1.489690e-07
84573,-0.012005,0.273036,-0.080949,0.021493,0.135640,0.547582,0.081690,-0.215371,-0.036512,0.086712,...,0.000000,0.0,0.000000,0.000000,1.668599e-07,1.668630e-07,1.524120e-07,1.524344e-07,0.0,1.563888e-07
84574,-0.016502,0.280284,-0.064395,0.018897,0.143809,0.553309,0.075904,-0.210710,-0.031080,0.090374,...,0.000000,0.0,0.000000,0.000000,1.589433e-07,1.589462e-07,1.451809e-07,1.452022e-07,0.0,1.489690e-07


## Supervised attack classification (Fusion)

We now train a supervised classifier on the fused features to predict multi-class attack labels:
- Features: embeddings + raw numeric features from `df_fuse_train`/`df_fuse_test` (without `Label`, `Attacks`).
- Target: `Attacks` (encoded integer classes from earlier `LabelEncoder`).
- Model: HistGradientBoostingClassifier (fast, strong on tabular data). Class imbalance handled via per-sample weights.
- Metrics: macro F1, per-class report, and confusion matrix.

In [34]:
# Prepare supervised train/test for attack classification
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import class_weight
import pickle

# Build train features/targets from already prepared fused DataFrames
X_sup_train = df_fuse_train.drop(columns=["Label", "Attacks"]).copy()
y_sup_train = df_fuse_train["Attacks"].copy()

X_sup_test = df_fuse_test.drop(columns=["Label", "Attacks"]).copy()
y_sup_test = df_fuse_test["Attacks"].copy()

# Compute sample weights to mitigate class imbalance on training set
classes = np.unique(y_sup_train)
class_w = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes, y=y_sup_train
)
class_to_w = {c: w for c, w in zip(classes, class_w)}
sample_weight = y_sup_train.map(class_to_w).values

# Feature names to all str
X_sup_train.columns = X_sup_train.columns.map(str)
X_sup_test.columns = X_sup_test.columns.map(str)

## Classification Methods (Fused Features)

Now we'll compare multiple classification algorithms on the fused features:

In [35]:
# Random Forest with Grid Search
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]
    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            rf_clf = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            rf_clf.fit(X_sup_train, y_sup_train)
            y_pred_rf = rf_clf.predict(X_sup_test)
            rf_f1 = f1_score(y_sup_test, y_pred_rf, average='macro')
            
            if rf_f1 > score:
                score = rf_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = rf_clf
                # Save best model
                with open('best_rf_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(rf_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_rf_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Random Forest: {str(e)}")

RANDOM FOREST CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [01:19<00:00,  8.80s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 20}
Best Macro F1: 0.5146
Model saved to: best_rf_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.2754    0.0333    0.0595      1380
           1     0.1488    0.4309    0.2213      1302
           2     0.9997    0.9935    0.9966     27542
           3     0.3421    0.1384    0.1971      3476
           4     0.7865    0.8049    0.7956     18932
           5     0.8272    0.7469    0.7850     13386
           6     0.9526    0.4849    0.6427      9936
           7     0.8347    0.7993    0.8166      7668
           8     0.0692    0.6379    0.1248       856
           9     0.4860    0.5306    0.5073        98

    accuracy                         0.7713     84576
   macro avg     0.5722    0.5601    0.5146     84576
weighted avg     0.8422    0.7713    0.7907     84576



In [36]:
# XGBoost with Grid Search
try:
    from xgboost import XGBClassifier
    import pickle
    
    print("="*60)
    print("XGBOOST CLASSIFIER (Fused Features)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        try:
            xgb_clf = XGBClassifier(
                n_estimators=n_est,
                max_depth=depth,
                learning_rate=lr,
                random_state=13,
                tree_method='hist', 
                device="cuda"
            )
            
            xgb_clf.fit(X_sup_train, y_sup_train, sample_weight=sample_weight)
            y_pred_xgb = xgb_clf.predict(X_sup_test)
            xgb_f1 = f1_score(y_sup_test, y_pred_xgb, average='macro')
            
            if xgb_f1 > score:
                score = xgb_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth,
                    'learning_rate': lr
                }
                best_model = xgb_clf
                # Save best model
                xgb_clf.save_model('best_xgb_classifier_fused.json')
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_xgb_classifier_fused.json\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed. XGBoost might not support your GPU.")
    
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

XGBOOST CLASSIFIER (Fused Features)
Testing 27 configurations...


  0%|          | 0/27 [00:00<?, ?it/s]

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:02:00] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
100%|██████████| 27/27 [12:10<00:00, 27.04s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6633
Model saved to: best_xgb_classifier_fused.json

              precision    recall  f1-score   support

           0     0.1625    0.1572    0.1599      1380
           1     0.1589    0.7104    0.2597      1302
           2     0.9838    0.9944    0.9891     27542
           3     0.3767    0.3720    0.3743      3476
           4     0.8991    0.8126    0.8537     18932
           5     0.9405    0.8357    0.8850     13386
           6     0.9634    0.8446    0.9001      9936
           7     0.8785    0.7996    0.8372      7668
           8     0.5498    0.8645    0.6721       856
           9     0.6636    0.7449    0.7019        98

    accuracy                         0.8481     84576
   macro avg     0.6577    0.7136    0.6633     84576
weighted avg     0.8902    0.8481    0.8645     84576



In [37]:
# CatBoost with Grid Search (Expanded, no l2_leaf_reg/border_count, fixed best score logic)
try:
    from catboost import CatBoostClassifier
    
    print("Starting CatBoost Grid Search (Expanded)...")
    
    # Expanded hyperparameter grid
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    print(f"Testing {len(params)} configurations...")
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'  # Use 'CPU' if GPU not available
            )
            
            cat_clf.fit(X_sup_train, y_sup_train)
            y_pred_cat = cat_clf.predict(X_sup_test)
            cat_f1 = f1_score(y_sup_test, y_pred_cat, average='macro')
            
            if cat_f1 > best_score:
                best_score = cat_f1
                best_params = {
                    'iterations': iterations,
                    'depth': depth,
                    'learning_rate': lr
                }
                best_model = cat_clf
                # Save the best model with unique name for fused features
                best_model_path = "best_catboost_classifier_fused.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("BEST CATBOOST HYPERPARAMETERS (Fused Features):")
    print(best_params)
    print(f"Best Macro F1: {best_score:.4f}")
    print("="*60)
    if best_model_path:
        print(f"\nBest CatBoost model saved to: {best_model_path}")
    
    # Final evaluation
    y_pred_best = best_model.predict(X_sup_test)
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):")
    print("="*60)
    print(classification_report(y_sup_test, y_pred_best, digits=4))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")

Starting CatBoost Grid Search (Expanded)...
Testing 27 configurations...


100%|██████████| 27/27 [08:37<00:00, 19.17s/it]


BEST CATBOOST HYPERPARAMETERS (Fused Features):
{'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.5986

Best CatBoost model saved to: best_catboost_classifier_fused.cbm

FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):
              precision    recall  f1-score   support

           0     0.1537    0.1587    0.1561      1380
           1     0.1558    0.5069    0.2384      1302
           2     0.9999    0.9940    0.9970     27542
           3     0.2581    0.4517    0.3285      3476
           4     0.9028    0.7326    0.8088     18932
           5     0.8971    0.7757    0.8320     13386
           6     0.9425    0.7952    0.8626      9936
           7     0.8427    0.8183    0.8304      7668
           8     0.3318    0.9404    0.4906       856
           9     0.3038    0.8061    0.4413        98

    accuracy                         0.8175     84576
   macro avg     0.5788    0.6980    0.5986     84576
weighted avg     0.8760    0.8175    0.8392     84

In [38]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]

    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            et_clf = ExtraTreesClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            et_clf.fit(X_sup_train, y_sup_train)
            y_pred_et = et_clf.predict(X_sup_test)
            et_f1 = f1_score(y_sup_test, y_pred_et, average='macro')
            
            if et_f1 > score:
                score = et_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = et_clf
                # Save best model
                with open('best_et_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(et_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_et_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Extra Trees: {str(e)}")

EXTRA TREES CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [00:51<00:00,  5.73s/it]



Best Parameters: {'n_estimators': 200, 'max_depth': 30}
Best Macro F1: 0.4730
Model saved to: best_et_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.1368    0.1130    0.1238      1380
           1     0.1399    0.1743    0.1552      1302
           2     0.9989    0.9919    0.9954     27542
           3     0.3034    0.1001    0.1506      3476
           4     0.7108    0.8354    0.7680     18932
           5     0.8136    0.7088    0.7576     13386
           6     0.9115    0.5119    0.6556      9936
           7     0.8468    0.7703    0.8067      7668
           8     0.0562    0.4463    0.0998       856
           9     0.3265    0.1633    0.2177        98

    accuracy                         0.7655     84576
   macro avg     0.5244    0.4815    0.4730     84576
weighted avg     0.8148    0.7655    0.7780     84576



## Raw Features Classification (Comparison Baseline)

Now we'll train classifiers on **raw features only** (without graph embeddings) to compare the benefit of multimodal fusion:

In [39]:
# Prepare raw features (without embeddings)
# Use only the original numeric features from X_train/X_test

X_raw_train = X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_train = y_train["Attack"].copy()

X_raw_test = X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_test = y_test["Attack"].copy()

# Encode labels
y_raw_train_encoded = lab_enc.transform(y_train["Attack"])
y_raw_test_encoded = lab_enc.transform(y_test["Attack"])

# Compute sample weights
classes_raw = np.unique(y_raw_train_encoded)
class_w_raw = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_raw, y=y_raw_train_encoded
)
class_to_w_raw = {c: w for c, w in zip(classes_raw, class_w_raw)}
sample_weight_raw = pd.Series(y_raw_train_encoded).map(class_to_w_raw).values

# Feature names to str
X_raw_train.columns = X_raw_train.columns.map(str)
X_raw_test.columns = X_raw_test.columns.map(str)

print(f"Raw feature shape - Train: {X_raw_train.shape}, Test: {X_raw_test.shape}")
print(f"Number of classes: {len(np.unique(y_raw_train_encoded))}")

Raw feature shape - Train: (98669, 39), Test: (42288, 39)
Number of classes: 10


In [40]:
# Raw Features - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Raw Features Only)")

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = rf_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_raw.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

RANDOM FOREST (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:22<00:00,  2.49s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 20}
Best Macro F1: 0.6009
Model saved to: best_rf_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.1573    0.8565    0.2658       690
           1     0.1874    0.1966    0.1919       651
           2     0.9993    0.9926    0.9959     13771
           3     0.5340    0.2980    0.3826      1738
           4     0.9012    0.7799    0.8362      9466
           5     0.9233    0.7895    0.8512      6693
           6     0.9620    0.8198    0.8852      4968
           7     0.8169    0.8135    0.8152      3834
           8     0.3498    0.8294    0.4920       428
           9     0.1797    0.7959    0.2932        49

    accuracy                         0.8314     42288
   macro avg     0.6011    0.7172    0.6009     42288
weighted avg     0.8915    0.8314    0.8525     42288



In [41]:
# Raw Features - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Raw Features Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='cuda'
        )
        
        xgb_clf.fit(X_raw_train, y_raw_train_encoded, sample_weight=sample_weight_raw)
        y_pred = xgb_clf.predict(X_raw_test)
        f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_raw.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_raw.json\n")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Raw Features Only)
Testing 27 configurations...


100%|██████████| 27/27 [03:15<00:00,  7.24s/it]


Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6703
Model saved to: best_xgb_classifier_raw.json

              precision    recall  f1-score   support

           0     0.1457    0.6275    0.2365       690
           1     0.1419    0.3041    0.1935       651
           2     0.9996    0.9939    0.9968     13771
           3     0.4929    0.3602    0.4162      1738
           4     0.9249    0.7881    0.8510      9466
           5     0.9447    0.8575    0.8990      6693
           6     0.9684    0.8504    0.9056      4968
           7     0.8538    0.8135    0.8332      3834
           8     0.5557    0.9439    0.6996       428
           9     0.5526    0.8571    0.6720        49

    accuracy                         0.8497     42288
   macro avg     0.6580    0.7396    0.6703     42288
weighted avg     0.9043    0.8497    0.8711     42288



In [42]:
# Raw Features - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Raw Features Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_raw_train, y_raw_train_encoded)
            y_pred = cat_clf.predict(X_raw_test)
            f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for raw features
                best_model_path = "best_catboost_classifier_raw.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Raw Features Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [02:31<00:00,  5.60s/it]


Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.6163

Best CatBoost model saved to: best_catboost_classifier_raw.cbm
              precision    recall  f1-score   support

           0     0.1538    0.7362    0.2544       690
           1     0.1481    0.2934    0.1968       651
           2     0.9999    0.9922    0.9960     13771
           3     0.3975    0.3516    0.3731      1738
           4     0.9229    0.7191    0.8083      9466
           5     0.9152    0.8050    0.8566      6693
           6     0.9472    0.8126    0.8748      4968
           7     0.8479    0.8200    0.8337      3834
           8     0.3823    0.9603    0.5469       428
           9     0.2678    1.0000    0.4224        49

    accuracy                         0.8232     42288
   macro avg     0.5982    0.7490    0.6163     42288
weighted avg     0.8905    0.8232    0.8478     42288



In [43]:
# Raw Features - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Raw Features Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = et_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_raw.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

EXTRA TREES (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:11<00:00,  1.29s/it]


Best Parameters: {'n_estimators': 200, 'max_depth': 30}
Best Macro F1: 0.6463
Model saved to: best_et_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.1548    0.8174    0.2603       690
           1     0.1652    0.1997    0.1808       651
           2     0.9984    0.9937    0.9960     13771
           3     0.5221    0.3199    0.3967      1738
           4     0.9063    0.7966    0.8479      9466
           5     0.9311    0.8253    0.8750      6693
           6     0.9623    0.8430    0.8987      4968
           7     0.8621    0.7890    0.8239      3834
           8     0.4538    0.9299    0.6100       428
           9     0.4362    0.8367    0.5734        49

    accuracy                         0.8431     42288
   macro avg     0.6392    0.7351    0.6463     42288
weighted avg     0.8982    0.8431    0.8631     42288



## Embeddings Only Classification (Graph Features)

Now we'll train classifiers on **embeddings only** (graph features without raw features) to isolate the value of graph-based learning:

In [44]:
# Prepare embeddings-only features (graph features without raw data)
# Extract only the embedding columns (first 256 dimensions from graph encoder)

# From the fused dataframes, extract only embedding columns
# df_fuse_train has: [0-255] = embeddings, [256+] = raw features
num_embedding_dims = 256  # Based on the DGI encoder output dimension

X_emb_train = df_fuse_train.iloc[:, :num_embedding_dims].copy()
y_emb_train = df_fuse_train["Attacks"].copy()

X_emb_test = df_fuse_test.iloc[:, :num_embedding_dims].copy()
y_emb_test = df_fuse_test["Attacks"].copy()

# Compute sample weights
classes_emb = np.unique(y_emb_train)
class_w_emb = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_emb, y=y_emb_train
)
class_to_w_emb = {c: w for c, w in zip(classes_emb, class_w_emb)}
sample_weight_emb = y_emb_train.map(class_to_w_emb).values

# Feature names to str
X_emb_train.columns = X_emb_train.columns.map(str)
X_emb_test.columns = X_emb_test.columns.map(str)

print(f"Embeddings shape - Train: {X_emb_train.shape}, Test: {X_emb_test.shape}")
print(f"Number of classes: {len(np.unique(y_emb_train))}")
print(f"Using only graph embeddings (no raw features)")

Embeddings shape - Train: (197338, 256), Test: (84576, 256)
Number of classes: 10
Using only graph embeddings (no raw features)


In [45]:
# Embeddings Only - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_emb_train, y_emb_train)
    y_pred = rf_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

RANDOM FOREST (Embeddings Only)
Testing 9 configurations...


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:34<00:00,  3.84s/it]


Best Parameters: {'n_estimators': 100, 'max_depth': 20}
Best Macro F1: 0.1896
Model saved to: best_rf_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0558    0.5330    0.1010      1302
           2     1.0000    0.9869    0.9934     27542
           3     0.0000    0.0000    0.0000      3476
           4     0.3831    0.0791    0.1312     18932
           5     0.2696    0.1962    0.2272     13386
           6     0.3296    0.4860    0.3928      9936
           7     0.1592    0.0121    0.0225      7668
           8     0.0126    0.1086    0.0226       856
           9     0.0029    0.2551    0.0057        98

    accuracy                         0.4379     84576
   macro avg     0.2213    0.2657    0.1896     84576
weighted avg     0.5082    0.4379    0.4388     84576




/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capit

In [46]:
# Embeddings Only - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Embeddings Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='gpu'
        )
        
        xgb_clf.fit(X_emb_train, y_emb_train, sample_weight=sample_weight_emb)
        y_pred = xgb_clf.predict(X_emb_test)
        f1 = f1_score(y_emb_test, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_embeddings.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_embeddings.json\n")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Embeddings Only)
Testing 27 configurations...


100%|██████████| 27/27 [08:22<00:00, 18.60s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.05}
Best Macro F1: 0.1948
Model saved to: best_xgb_classifier_embeddings.json

              precision    recall  f1-score   support

           0     0.0072    0.0145    0.0096      1380
           1     0.0531    0.2043    0.0843      1302
           2     0.9816    0.9869    0.9842     27542
           3     0.0662    0.0400    0.0499      3476
           4     0.4051    0.0691    0.1180     18932
           5     0.2545    0.1987    0.2232     13386
           6     0.2947    0.4607    0.3595      9936
           7     0.1325    0.0627    0.0851      7668
           8     0.0187    0.0736    0.0298       856
           9     0.0023    0.2551    0.0046        98

    accuracy                         0.4342     84576
   macro avg     0.2216    0.2366    0.1948     84576
weighted avg     0.5011    0.4342    0.4360     84576



In [47]:
# Embeddings Only - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Embeddings Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_emb_train, y_emb_train)
            y_pred = cat_clf.predict(X_emb_test)
            f1 = f1_score(y_emb_test, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for embeddings
                best_model_path = "best_catboost_classifier_embeddings.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Embeddings Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [06:31<00:00, 14.50s/it]


Best Parameters: {'iterations': 200, 'depth': 8, 'learning_rate': 0.05}
Best Macro F1: 0.2001

Best CatBoost model saved to: best_catboost_classifier_embeddings.cbm
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0555    0.4662    0.0992      1302
           2     1.0000    0.9869    0.9934     27542
           3     0.0743    0.0768    0.0755      3476
           4     0.3804    0.1486    0.2137     18932
           5     0.2734    0.1271    0.1736     13386
           6     0.4720    0.2361    0.3148      9936
           7     0.1302    0.0788    0.0982      7668
           8     0.0149    0.2325    0.0280       856
           9     0.0022    0.1429    0.0044        98

    accuracy                         0.4225     84576
   macro avg     0.2403    0.2496    0.2001     84576
weighted avg     0.5254    0.4225    0.4496     84576




/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capit

In [48]:
# Embeddings Only - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_emb_train, y_emb_train)
    y_pred = et_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

EXTRA TREES (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [00:23<00:00,  2.63s/it]


Best Parameters: {'n_estimators': 100, 'max_depth': 10}
Best Macro F1: 0.1839
Model saved to: best_et_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1380
           1     0.0526    0.4793    0.0948      1302
           2     1.0000    0.9869    0.9934     27542
           3     0.0000    0.0000    0.0000      3476
           4     0.3745    0.0678    0.1148     18932
           5     0.2713    0.1780    0.2150     13386
           6     0.2975    0.5061    0.3748      9936
           7     0.1663    0.0104    0.0196      7668
           8     0.0118    0.0689    0.0201       856
           9     0.0030    0.3367    0.0060        98

    accuracy                         0.4336     84576
   macro avg     0.2177    0.2634    0.1839     84576
weighted avg     0.5034    0.4336    0.4307     84576




/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capit

## Summary Comparison Table

Create a comparison table of all methods (Fused vs Raw features):

In [49]:
# Create a summary comparison table
# NOTE: After running all cells above, manually collect the F1 scores and create a comparison

print("="*95)
print("PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES")
print("="*95)
print("\nInstructions:")
print("1. Run all cells above to get F1 scores for each method")
print("2. Record the best Macro F1 score for each method")
print("3. Compare three approaches:")
print("   - Fused Features: Embeddings + Raw (Multimodal)")
print("   - Embeddings Only: Graph features only")
print("   - Raw Features: Traditional features only")
print("\nExpected format:")
print("-" * 95)
print(f"{'Method':<20} {'Fused (Emb+Raw)':<25} {'Embeddings Only':<25} {'Raw Only':<25}")
print("-" * 95)
print(f"{'Random Forest':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'XGBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'CatBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'Extra Trees':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print("-" * 95)
print("\nAnalysis:")
print("- Fused features should show best performance (multimodal learning)")
print("- Embeddings capture structural/graph patterns")
print("- Raw features provide traditional statistical information")
print("- Compare to understand the contribution of each modality")

PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES

Instructions:
1. Run all cells above to get F1 scores for each method
2. Record the best Macro F1 score for each method
3. Compare three approaches:
   - Fused Features: Embeddings + Raw (Multimodal)
   - Embeddings Only: Graph features only
   - Raw Features: Traditional features only

Expected format:
-----------------------------------------------------------------------------------------------
Method               Fused (Emb+Raw)           Embeddings Only           Raw Only                 
-----------------------------------------------------------------------------------------------
Random Forest        [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
XGBoost              [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
CatBoost             [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
Extra Trees          [INSERT SCORE]           

## Key Insights & Interpretation

After running all experiments, analyze the results to answer:

1. **Which approach performs best overall?**
   - Fused features (multimodal) should ideally outperform single modalities
   - Compare the magnitude of improvements

2. **What is the value of graph embeddings?**
   - Compare Embeddings-only vs Raw-only to see if graph structure helps
   - If embeddings alone beat raw features, graph learning is beneficial

3. **Is multimodal fusion effective?**
   - Compare Fused vs (Embeddings + Raw separately)
   - Synergy should provide additional gains beyond individual modalities

4. **Which classifier is most suitable?**
   - Identify the best performing algorithm for each feature type
   - Consider computational cost vs accuracy trade-offs

5. **Feature contribution analysis:**
   - If Fused ≈ Embeddings > Raw: Graph structure dominates
   - If Fused ≈ Raw > Embeddings: Traditional features dominate
   - If Fused > Both: True synergy from multimodal learning